# An overview of `astropy.uncertainty`:

A relatively recent addition to `astropy` is the `astropy.uncertainty` sub-package.  Its primary purpose is to represent the *uncertainties* of Quantities in a way that allows relatively straightforward application of error-propogation rules.

Some important caveats for `astropy.uncertainty`: it is *not* intended to be a fully-featured replacement for thorough statistical analysis. For that you will want to use more statistical modeling approaches like combining statistically-oriented fitting tools with `astropy` pieces for just the astro-specific parts. E.g., you might use the `emcee` MCMC sampler with `astropy.modeling` astronomy-specific models implementing the likelihood function.  `astropy.uncertainty` is instead meant to provide a vehicle by which you can store uncertainties, and follow the basic error propogation rules when your science case does not require full statistical modeling.

Moreover, it is a newer sub-package.  While we do not anticipate major changes, it is possible some of the interface will evolve in future versions of astropy.

### *Note: This notebook is a copy of the tutorial notebook with some redundant cells omitted and with exercise solutions filled in*

### Preliminary imports

We start by importing some general packages we will need below:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

from astropy.visualization import quantity_support
quantity_support()

from IPython import display

In [ ]:
from astropy import units as u

from astropy import uncertainty

# Recording uncertainties of data

The first use case for `uncertainty` is simply storing uncertainties *with* a quantity of interest. Lets start with a concrete and fairly typical use case: magnitudes of some set of objects drawn from an existing paper. As an example, consider the two galaxies highlighted in this paper: https://doi.org/10.1088/2041-8205/798/1/L21, which have apparent $r$-band magnitudes given with standard symmetric error bars. The typical assumption is that these values are to be thought of as having Gaussian/Normal uncertainties with the uncertainty being the $\sigma$ of the Gaussian. Lets start by trying to represent just one of these (Pisces A) using `uncertainty`:

In [ ]:
piscA_mr = uncertainty.normal(17.35*u.mag, std=0.05*u.mag, n_samples=10000)
piscA_mr

In [ ]:
type(piscA_mr)

It is immediately apparent something has happened here beyond just recording the value and its $\sigma$.  `astropy.uncertainty` uses a Monte Carlo representation of the quantities it stores.  So when we called `uncertainty.normal`, we created a normal distribution of numbers with the given parameters.  This is why `n_samples` is required: only you, the user, knows how careful you want to be in modeling the uncertainties.  The standard choice of 10000 is reasonable, as the error in your uncertainty-related parameters generally go like $\sqrt(N_{\rm samples})$, so with 10000 samples you can trust your estimtes *on the uncertainties* to 1%.

To illustrate what has happened, lets use some of the convenience parameters `QuantityDistribution` provides:

In [ ]:
piscA_mr.pdf_mean()

In [ ]:
piscA_mr.pdf_std()

In [ ]:
piscA_mr.pdf_median()

The above compute the mean, standard deviation, and median of the *samples* from the distribution. It is apparent that this reproduces the input we gave but only to ~1%, as expected from the number of samples.  We can also access the samples directly, as needed to produce, for example, a plot of the distribution:

In [ ]:
plt.hist(piscA_mr.distribution, bins='auto', density=True, histtype='step')

plt.axvline(piscA_mr.pdf_mean(), color='k')
plt.axvline(piscA_mr.pdf_mean() + piscA_mr.pdf_std(), color='k', ls=':')
plt.axvline(piscA_mr.pdf_mean() - piscA_mr.pdf_std(), color='k', ls=':');

Now we can represent both galaxies from this paper the same way,  letting us do quick convenient operations over them:

In [ ]:
piscB_mr = uncertainty.normal(17.18*u.mag, std=0.07*u.mag, n_samples=10000)

for distr in [piscA_mr, piscB_mr]:
    plt.hist(distr.distribution, bins='auto', density=True, histtype='step')
    
    plt.axvline(distr.pdf_mean(), color='k')
    plt.axvline(distr.pdf_mean() + distr.pdf_std(), color='k', ls=':')
    plt.axvline(distr.pdf_mean() - distr.pdf_std(), color='k', ls=':')


This on its own is enough to ask simple statistical questions using just the samples.  For example: what is the probability that Pisces A is brighter than Pisces B?

In [ ]:
gridA, gridB = np.meshgrid(piscA_mr.distribution, piscB_mr.distribution)
np.sum(gridA < gridB) / gridA.size

Depending on your computer's speed, you may have seen a noticeable amount of time passing for that computation. This is because the `meshgrid` function created an array element for every possible pair in the two distributions, which is $10000^2 = 10^8$, meaning it had to create two 100 million element array.  This probably used a few GB of your computer's memory, a not-insignificant fraction of what you have. This is one of the "gotchas" to be aware of with the monte carlo method: when doing operations that involve multiple distributions you often need to compare all combinations, which quickly becomes exponentially large and overwhelms your computer.

Just in case, we delete the large variables we created above. (so that if your computer has limited memory it won't become a problem later):

In [ ]:
del gridA, gridB

A reasonable compromise to get around this is to just use the fact that the two distributions are independent and compare them element-wise:

In [ ]:
np.sum(piscA_mr.distribution < piscB_mr.distribution) / piscA_mr.n_samples

This still gets a similar answer as the more complete version, although it is different at the percent level as expected for $10^4$ $n_{\rm samples}$

### Exercise

What is the probability that Pisces A and Pisces B are within .1 mags of each other?

In [ ]:
gridA, gridB = np.meshgrid(piscA_mr.distribution, piscB_mr.distribution)
np.sum(np.abs(gridA - gridB) < 0.1*u.mag) / gridA.size

In [ ]:
# or at lower precision but a lot faster/less memory-intensive
np.sum(np.abs(piscA_mr.distribution - piscB_mr.distribution) < 0.1*u.mag) / piscA_mr.n_samples

## Array distributions

Thus far this has not done much that you couldn't do by just making the Gaussians yourself.  But one of the use-cases for uncertainties as making it easier to wrap up multiple values as arrays that each contain distributions: 

In [ ]:
galaxies_mr = uncertainty.normal([17.35, 17.18]*u.mag, std=[.05, .07]*u.mag, n_samples=10000)
galaxies_mr

In [ ]:
galaxies_mr.shape

Despite holding a large number of samples, this looks like a single quantity of length 2.  We can still get at the samples though:

In [ ]:
galaxies_mr.distribution.shape

In [ ]:
for dist in galaxies_mr.distribution:
    plt.hist(dist, bins='auto', density=True, histtype='step')

With a bit of string-processing and Jupyter notebook ticks, we can also use this to produce some nicer-looking quantities:

In [ ]:
for mr in galaxies_mr:
    mean = mr.pdf_mean()
    std = mr.pdf_std()
    lstr = '${mean:.2f} \pm {std:.2f}$'
    
    # or equivalently, a one-liner using Python f-strings:
    lstr = f'${mr.pdf_mean():.2f} \pm {mr.pdf_std():.2f}$'

    display.display(display.Latex(lstr))

## Using Distributions as Quantities

But the real power of `Distribution`s is their ability to be treated just like ordinary quantities. (To refresh yourself on `Quantities` you can have a look at the [units and quantitites notebook](../03-UnitsQuantities/Astropy_Units.ipynb).)  For example, we can represent *both* galaxies from this paper as a single `Distribution`:

While the above provides some conveniences, more utility comes from treating these as quantities the way you would any other quantity.  For example, suppose we wanted to convert these magnitudes to fluxes following the standard Pogson formulation of magnitudes:

$m = -2.5 \log_{10}(f)$

(Note there are some more convenient ways to handle this conversion in `astropy` - see the [docs section on this in astropy.units](https://docs.astropy.org/en/stable/units/logarithmic_units.html), but here we do it by-hand to illustrate how to use `uncertainty` in a more general way.)

In [ ]:
galaxies_rflux = 10**(galaxies_mr/(-2.5*u.mag)) * u.ABflux
galaxies_rflux

In [ ]:
galaxies_rflux.pdf_mean()

In [ ]:
galaxies_rflux.pdf_std()

In [ ]:
for dist in galaxies_rflux.distribution:
    plt.hist(dist, bins='auto', density=True, histtype='step')

Close inspection of this distribution shows that it is no longer quite Gaussian, as there is an extended tail to higher fluxes.  This is more apparent if we artifically inflate the magnitude uncertainty by a factor of 10:

In [ ]:
galaxies_mr_inflated_uncertainty = uncertainty.normal([17.35, 17.18]*u.mag, std=[.5, .7]*u.mag, n_samples=10000)
galaxies_rflux_inflated_uncertainty = 10**(galaxies_mr_inflated_uncertainty/(-2.5*u.mag)) * u.ABflux
for dist in galaxies_rflux_inflated_uncertainty.distribution:
    plt.hist(dist, bins='auto', density=True, histtype='step',)

And similarly, the error bars now clearly need to be assymetric, as demonstrated by comparing the standard deviation to the 16% / 84% tails of the distribution:

In [ ]:
galaxies_rflux_inflated_uncertainty.pdf_std()

In [ ]:
galaxies_rflux_inflated_uncertainty.pdf_percentiles(16) - galaxies_rflux_inflated_uncertainty.pdf_median()

In [ ]:

galaxies_rflux_inflated_uncertainty.pdf_percentiles(84) - galaxies_rflux_inflated_uncertainty.pdf_median()

In [ ]:
galaxies_rflux_inflated_uncertainty

In [ ]:
for f in galaxies_rflux_inflated_uncertainty:
    lower, mid, upper = f.pdf_percentiles([16, 50, 84]).value/1e-7
    lstr = f'${mid:.2} ^ {{ +{upper-mid:.2} }} _ {{ {lower-mid:.2} }} \\times 10^{-7}$'

    display.display(display.Latex(lstr))

## More complex manipulations with other Astropy functionality

While there is plenty to be done with quantities, `uncertainty` is also useful for more complex `astropy` objects.  We will illustrate this by using the `astropy.coordinates.SkyCoord` object.  This section assumes at least some familiarity with `coordinates`, so if you are confused by some of the coordinates-related operations, you may want to look at the [coordinates notebook](../04-Coordinates/astropy_coordinates.ipynb).

We need to import functionality from the other parts of astropy we will use:

In [ ]:
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time

Now, let's assume you are looking at an image from an imager on a telescope at the Cerro Tololo Inter-American Observatory (CTIO).  In that image, you have a star you are interested in for some reason.  You have measured the centroid of the star in alt/az coordinates (as observed from your site at a particular time), but the conditions were not stellar (get it?), and the seeing was significantly worse than an arcsec. So you conclude your uncertainty is about an arcsec. We can encode that by creating a relevant `SkyCoord`, but providing the star as a distribution instead of a raw quantity:

In [ ]:
alt = uncertainty.normal(50*u.deg, std=1*u.arcsec, n_samples=10000)
az = uncertainty.normal(128*u.deg, std=1*u.arcsec, n_samples=10000)
alt, az

Note `uncertainty` was perfectly happy to accept different units for the value and its `std`, and took care of the conversion for you.

We can visualize this uncertainty on-sky by just plotting the distribution and letting the density of points indicate to us the probability distribution:

In [ ]:
plt.subplot(aspect='equal')
plt.scatter(az.distribution, alt.distribution.to(u.deg).value, s=1, alpha=.25)

plt.xlabel('Azimuth [deg]')
plt.ylabel('Altitude [deg]')

Now imagine you want to compare this to some stars in a catalog to match them to your observation.  But this catalog is in equatorial coordinates (ICRS RA & Dec). We need to convert our observations to that sysyem:

In [ ]:
ctio = EarthLocation.of_site('CTIO')
# if the above line fails because of internet issues, uncomment the following line and comment the above - it should give the same answer.
#ctio = EarthLocation.from_geodetic(lon=-70.815*u.deg, lat=-30.16527777777777*u.deg, height=2215*u.m)

obstime = Time('2023-12-21T03:00:00', scale='utc')

In [ ]:
i = SkyCoord(ra=az, dec=alt, location=ctio, obstime=obstime)

In [ ]:
i.fk5

In [ ]:
i.altaz

In [ ]:
altaz = SkyCoord(alt=alt, az=az, frame='altaz', location=ctio, obstime=obstime)
icrs = altaz.transform_to('icrs')

Do'h!  Doesn't work.  Re-try with galactic:

In [ ]:
b = uncertainty.normal(50*u.deg, std=1*u.arcsec, n_samples=10000)
l = uncertainty.normal(128*u.deg, std=1*u.arcsec, n_samples=10000)
gal = SkyCoord(l=l, b=b, frame='galactic')

plt.subplot(aspect='equal')
plt.scatter(gal.icrs.ra.distribution, gal.icrs.dec.distribution.to(u.deg).value, s=1, alpha=.25)

plt.xlabel('Azimuth [deg]')
plt.ylabel('Altitude [deg]')

Ok that will work, carry on for now

# Beware the ghost of covariances past

In [ ]:
Feh = uncertainty.normal(-1.5, std=0.1, n_samples=10000)
afe = uncertainty.normal(0.5, std=0.1, n_samples=10000)

plt.scatter(Feh.distribution, afe.distribution, s=1, alpha=.25)

In [ ]:
fe_abund = uncertainty.normal(-1.5, std=0.1, n_samples=10000)
al_abund = uncertainty.normal(-1.0, std=0.1, n_samples=10000)

afe = al_abund - fe_abund

plt.scatter(fe_abund.distribution, afe.distribution, s=1, alpha=.25)

### Exercises

 Description of exercise

In [ ]:
solution = 'correct'

## Wrap-up

This tutorial covers a lot of material, but `astropy.uncertainty` has even more functionality that we were unable to cover in this workshop. For documentation on other features of `astropy.uncertainty`, check out [the astropy.uncertainty section of the Astropy documentation](http://astropy.readthedocs.org/en/stable/uncertainty/index.html).